**Here we perform the training loops**

In [1]:
from data_frame_manager.data_frame_manager import DataFrameManager
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
# import xgboost as xgb
import pandas as pd


DATASET_COLUMNS = ["text", "label"]#["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
MODEL_NAME = 'roberta'
NUM_CLASSES = 3


In [11]:
dataFrameManage = DataFrameManager()

train_df = dataFrameManage.load_dataframe(filepath="Data/twitter-datasets/preprocessed/train_preprocessed.csv", encoding=DATASET_ENCODING, preprocess=False)
test_df = dataFrameManage.load_dataframe(filepath="Data/twitter-datasets/preprocessed/test_preprocessed.csv", encoding=DATASET_ENCODING, preprocess=False)

encode_map = {"NEGATIVE" : 0, "POSITIVE" : 1}


train_labels = train_df["target"].map(encode_map).to_list()
test_labels = test_df["target"].map(encode_map).to_list()

# CountVectorizer

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# We only keep the 5000 most frequent words, both to reduce the computational cost and reduce overfitting
vectorizer = CountVectorizer(max_features=5000)

# Important: we call fit_transform on the training set, and only transform on the validation set
X_train = vectorizer.fit_transform(train_df["text"].to_list())
X_val = vectorizer.transform(test_df["text"].to_list())

Y_train = train_labels
Y_val = test_labels

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1e5, max_iter=100)
model.fit(X_train, Y_train)

/Users/filippoficarra/Documents/GitHub/Computational-Intelligence-Lab-Sentiment-analysis/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100000.0)

In [14]:
Y_train_pred = model.predict(X_train)
Y_val_pred = model.predict(X_val)

train_accuracy = accuracy_score(Y_train_pred, Y_train)
val_accuracy = accuracy_score(Y_val_pred, Y_val)
print(f'Accuracy (training set): {train_accuracy:.05f}')
print(f'Accuracy (validation set): {val_accuracy:.05f}')

Accuracy (training set): 0.79841
Accuracy (validation set): 0.79541
